In [61]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import random

Q1

In [62]:
def y(x):
    return 2*x**2+4

In [63]:
class QuadraticNet(nn.Module):
    def __init__(self):
        super(QuadraticNet, self).__init__()
        self.hidden = nn.Linear(1, 10)
        self.output = nn.Linear(10, 1)

    def forward(self, x):
        x = self.hidden(x)  # linear activation 
        x = self.output(x)
        return x

model = QuadraticNet()
print(model)

QuadraticNet(
  (hidden): Linear(in_features=1, out_features=10, bias=True)
  (output): Linear(in_features=10, out_features=1, bias=True)
)


In [64]:
x = torch.linspace(0, 10, steps=101).unsqueeze(1)
y = y(x)

In [65]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training
epochs = 2000

for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = model(x**2)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()

    if epoch % 200 == 0:
        print(f"Epoch {epoch}, Loss = {loss.item():.6f}")

# Test x = 3
test_x = torch.tensor([[3.0]])
pred_y = model(test_x**2)

true_y = 2 * 3**2 + 4

print("\nTest result:")
print("Predicted y =", pred_y.item())
print("True y      =", true_y)



Epoch 0, Loss = 6412.703125
Epoch 200, Loss = 3.924824
Epoch 400, Loss = 1.441833
Epoch 600, Loss = 0.371898
Epoch 800, Loss = 0.067572
Epoch 1000, Loss = 0.008536
Epoch 1200, Loss = 0.000736
Epoch 1400, Loss = 0.000042
Epoch 1600, Loss = 0.000002
Epoch 1800, Loss = 0.000000

Test result:
Predicted y = 21.999971389770508
True y      = 22


Q2

In [69]:
data = np.load(r"D:\KAUST\semster2\ML_IN_GEO\SiesmicEventsClassification_Normalized.npz")

In [70]:
X = data['data']  
y = data['label']  

print(X.shape)
print(y.shape)


(1000, 6000)
(1000,)


In [ ]:
#convert data into frequency domain
X_freq = np.fft.fft(X, axis=1) 
X_freq_real = X_freq.real
X_freq_imag = X_freq.imag
X_freq_combined = np.concatenate([X_freq_real, X_freq_imag], axis=1)

In [73]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_freq_combined, y, test_size=0.2, random_state=42, shuffle=True
)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor  = torch.tensor(y_test, dtype=torch.long)

In [74]:
class SeismicNet(nn.Module):
    def __init__(self, input_size, hidden_layers=[64, 32], output_size=2):
        super().__init__()
        layers = []
        last_size = input_size
        for h in hidden_layers:
            layers.append(nn.Linear(last_size, h))
            layers.append(nn.ReLU())
            last_size = h
        layers.append(nn.Linear(last_size, output_size))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# training function
def train_model(model, X_train, y_train, epochs=20, batch_size=32, lr=0.001):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    model.train()
    
    for epoch in range(epochs):
        permutation = torch.randperm(X_train.size()[0])
        correct = 0
        total = 0
        for i in range(0, X_train.size()[0], batch_size):
            indices = permutation[i:i+batch_size]
            batch_x, batch_y = X_train[indices], y_train[indices]
            
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            # calculate accuracy
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == batch_y).sum().item()
            total += batch_y.size(0)
        
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}/{epochs}, Accuracy: {epoch_acc:.4f}")
        
    return model

# testing model
def test_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        preds = torch.argmax(outputs, dim=1)
        correct = (preds == y_test).sum().item()
        acc = correct / y_test.size(0)
    return acc

In [75]:
import torch

# set config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


X_train_tensor = X_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
X_test_tensor  = X_test_tensor.to(device)
y_test_tensor  = y_test_tensor.to(device)

# ==============================
hidden_layer_configs = [
    # 1 hidden layer
    [16], [32], [64], [128], [256],

    # 2 hidden layer
    [32, 16], [64, 32], [128, 64], [256, 128], [128, 32], [64, 16],

    # 3 hidden layer
    [64, 32, 16], [128, 64, 32], [256, 128, 64], [128, 64, 16], [64, 32, 16],

    # 4 hidden layer
    [128, 64, 32, 16], [256, 128, 64, 32], [64, 64, 32, 16],
]

batch_sizes = [16, 32, 64]
learning_rates = [0.001, 0.005, 0.01]

best_acc = 0
best_config = None
best_batch = None
best_lr = None

input_size = X_train_tensor.shape[1]

# ==============================
# training & testing
# ==============================
for config in hidden_layer_configs:
    for batch_size in batch_sizes:
        for lr in learning_rates:
            print(f"\nTraining with hidden layers: {config}, batch size: {batch_size}, lr: {lr}")
            
            
            model = SeismicNet(input_size=input_size, hidden_layers=config)
            model = model.to(device)
            
            
            model = train_model(model, X_train_tensor, y_train_tensor,
                                epochs=20, batch_size=batch_size, lr=lr)
            
            
            acc = test_model(model, X_test_tensor, y_test_tensor)
            print(f"Test Accuracy: {acc:.4f}")
            
            
            if acc > best_acc:
                best_acc = acc
                best_config = config
                best_batch = batch_size
                best_lr = lr

print("\n=============================")
print("Best hidden layer config:", best_config)
print("Best batch size:", best_batch)
print("Best learning rate:", best_lr)
print("Best test accuracy:", best_acc)


Using device: cuda

Training with hidden layers: [16], batch size: 16, lr: 0.001
Epoch 1/20, Accuracy: 0.5075
Epoch 2/20, Accuracy: 0.8450
Epoch 3/20, Accuracy: 0.9613
Epoch 4/20, Accuracy: 0.9950
Epoch 5/20, Accuracy: 0.9975
Epoch 6/20, Accuracy: 0.9975
Epoch 7/20, Accuracy: 0.9975
Epoch 8/20, Accuracy: 0.9975
Epoch 9/20, Accuracy: 0.9975
Epoch 10/20, Accuracy: 0.9975
Epoch 11/20, Accuracy: 0.9975
Epoch 12/20, Accuracy: 0.9975
Epoch 13/20, Accuracy: 0.9975
Epoch 14/20, Accuracy: 0.9975
Epoch 15/20, Accuracy: 0.9975
Epoch 16/20, Accuracy: 0.9975
Epoch 17/20, Accuracy: 0.9975
Epoch 18/20, Accuracy: 0.9975
Epoch 19/20, Accuracy: 0.9975
Epoch 20/20, Accuracy: 0.9975
Test Accuracy: 0.4700

Training with hidden layers: [16], batch size: 16, lr: 0.005
Epoch 1/20, Accuracy: 0.4938
Epoch 2/20, Accuracy: 0.6925
Epoch 3/20, Accuracy: 0.8675
Epoch 4/20, Accuracy: 0.9350
Epoch 5/20, Accuracy: 0.9587
Epoch 6/20, Accuracy: 0.9663
Epoch 7/20, Accuracy: 0.9838
Epoch 8/20, Accuracy: 0.9788
Epoch 9/20, 

In [76]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import torch
# 
model.eval()

with torch.no_grad():
    outputs = model(X_test_tensor)          
    preds = torch.argmax(outputs, dim=1)   
preds_cpu = preds.cpu().numpy()
y_test_cpu = y_test_tensor.cpu().numpy()


In [78]:
cm = confusion_matrix(y_test_cpu, preds_cpu)
print("Confusion Matrix:")
print(cm)

precision = precision_score(y_test_cpu, preds_cpu, average='weighted')
recall = recall_score(y_test_cpu, preds_cpu, average='weighted')
f1 = f1_score(y_test_cpu, preds_cpu, average='weighted')

print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")


Confusion Matrix:
[[26 76]
 [30 68]]
Precision: 0.4682
Recall:    0.4700
F1 Score:  0.4432
